In [1]:
import pandas as pd
DATA_PATH = '../../data' #Insert path to data here
from sklearn.model_selection import train_test_split

**Si recuperano tutti i valori nel campo ‘city_or_county’ per i quali non è possibile determinare se si riferiscono a una ‘city’ o a una ‘county’.**

In [2]:
cities = pd.read_csv("../../data/uscities.csv")
county = pd.read_csv("../../data/county.csv")
df = pd.read_csv(f"{DATA_PATH}/cluster.csv")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df.date.dt.year
df = df[(df.year >= 2014) & (df.year <= 2017) ]

county_entries = df[df['city_or_county'].str.contains('county', case=False, na=False)]['city_or_county']
not_recognized = df[(~df.city_or_county.isin(cities.city)) & (~df.city_or_county.isin(county.name)) & (~df.city_or_county.isin(county_entries))].drop_duplicates(subset=['city_or_county'])

**Tramite latitude e longitude recuperiamo la relativa città**

In [3]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="prova")
def get_city(row):
    location = geolocator.reverse([row['latitude'], row['longitude']], exactly_one=True).raw['address']
    return location.get('city', None) #Restituisce null se la posizione non si riferisce ad una città

not_recognized['city'] = not_recognized.apply(get_city, axis=1)
not_recognized.to_csv(f"../data/not_recognized.csv", index=False)

**Recuperiamo solo le entry che contengono le città**

In [4]:
def normalize_city_name(df: pd.DataFrame):
    """
    Recupera i nomi delle città non presenti nel dataset uscities.csv e stabilisce la città in cui si è verificato l'incidente in base alle coordinate geografiche.
    Se la posizione non si riferisce ad una città, allora la entry viene rimossa
    """
    
    additional_cities = pd.read_csv("data/not_recognized.csv")[["city_or_county", "city"]].dropna()
    additional_cities[additional_cities.city_or_county != additional_cities.city]
    df2 = df[df.city_or_county.isin(additional_cities.city_or_county)]

    # Tutte le entry di df['city_or_county'] che sono presenti 
    # in additional_cities['city_or_county'] verranno sostituite con il corrispondente valore di additional_cities['city']
    dict_additional_cities = additional_cities.set_index('city_or_county')['city'].to_dict()
    df2['city_or_county'] = df2['city_or_county'].map(dict_additional_cities).fillna(df['city_or_county'])
    
    return df2


df1 = df[df.city_or_county.isin(cities.city)]
df2 = normalize_city_name(df)
final_df = pd.concat([df1, df2])

final_df.rename(columns={'city_or_county': 'city'}, inplace=True)
final_df.to_csv(f"data/time_Series_Data.csv", index=False)

/tmp/ipykernel_22369/4239925358.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['city_or_county'] = df2['city_or_county'].map(dict_additional_cities).fillna(df['city_or_county'])
